In [2]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
data = [['A','F'],
          ['A','D','E'],
          ['E', 'G'],
          ['A',' C','D'],
          ['A','F', 'G'],
          ['A','B','C','F','G'],
          ['A','B','C','D','G'],
          ['B','D','G'],
          ['A','C','E','F','G'],
           ['C','D']]
I = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
MinsupRatio = 3/10
MinsupValue = 3
print("1.3. FPGrowth alg")
print("Data: ", data)
print("Minsup ratio: ", MinsupRatio)
print("Minsup value: ", MinsupValue)
print("Unique el's: ", I)
print()
print("Start FPGrowthAlg with I and minsup value: ", I, ", ", MinsupValue)
from collections import defaultdict, namedtuple
def myFpgrowth(dataSet, minSup):
    items = defaultdict(lambda: 0)
    for d in dataSet:
        for item in d:
            items[item] += 1
    items = dict((item, support) for item, support in items.items())
    items = dict(sorted(items.items()))
    items = dict(sorted(items.items(), key=lambda i: i[1], reverse=True))
    for k in items.items(): 
        print("Add to F:", k)
    print(" ")
    print("Start R_FPGrowthAlg with Data, Add and gen_prefix: ", dataSet, ",",  "None")
    print("Result sorting unique: ", items)
    master = FPTree()
    for d in dataSet:
        d.sort(key=lambda v: items[v], reverse=True)
        master.add(d)
    print("FP Tree: root:", end="")
    master.disp()
    print(")")
    print()
    def findWithSuffix(tree, suffix):
        for item, nodes in tree.items():
            support = sum(n.count for n in nodes)
            if support >= minSup and item not in suffix:
                found_set = [item] + suffix
                yield (found_set, support)
                cond_tree = conditionalTreeFromPaths(tree.prefix_paths(item))            
                print("C-FP Tree: root:", end="")
                cond_tree.disp()
                print(")")
                for s in findWithSuffix(cond_tree, found_set):
                    yield s 
    for itemset in findWithSuffix(master, []):
        yield itemset

class FPTree(object):
    Route = namedtuple('Route', 'head tail')
    def __init__(self):
        self._root = FPNode(self, None, None)
        self._routes = {}
    @property
    def root(self):
        return self._root
    def add(self, transaction):
        point = self._root

        for item in transaction:
            next_point = point.search(item)
            if next_point:
                next_point.increment()
            else:
                next_point = FPNode(self, item)
                point.add(next_point)
                self._update_route(next_point)
            point = next_point
    def _update_route(self, point):
        try:
            route = self._routes[point.item]
            route[1].neighbor = point 
            self._routes[point.item] = self.Route(route[0], point)
        except KeyError:
            self._routes[point.item] = self.Route(point, point)
    def items(self):
        for item in self._routes.keys():
            yield (item, self.nodes(item))
    def nodes(self, item):
        try:
            node = self._routes[item][0]
        except KeyError:
            return
        while node:
            yield node
            node = node.neighbor
    def prefix_paths(self, item):
        def collect_path(node):
            path = []
            while node and not node.root:
                path.append(node)
                node = node.parent
            path.reverse()
            return path
        return (collect_path(node) for node in self.nodes(item))
    def disp(self):
        self._root.disp()
        
def conditionalTreeFromPaths(paths):
    tree = FPTree()
    condition_item = None
    items = set()
    for path in paths:
        if condition_item is None:
            condition_item = path[-1].item
        point = tree.root
        for node in path:
            next_point = point.search(node.item)
            if not next_point:
                items.add(node.item)
                count = node.count if node.item == condition_item else 0
                next_point = FPNode(tree, node.item, count)
                point.add(next_point)
                tree._update_route(next_point)
            point = next_point
    for path in tree.prefix_paths(condition_item):
        count = path[-1].count
        for node in reversed(path[:-1]):
            node._count += count
    return tree

class FPNode(object):
    def __init__(self, tree, item, count=1):
        self._tree = tree
        self._item = item
        self._count = count
        self._parent = None
        self._children = {}
        self._neighbor = None
        self._first = True
    def add(self, child):
        if not child.item in self._children:
            self._children[child.item] = child
            child.parent = self
    def search(self, item):
        try:
            return self._children[item]
        except KeyError:
            return None
    def __contains__(self, item):
        return item in self._children
    @property
    def tree(self):
        return self._tree
    @property
    def item(self):
        return self._item
    @property
    def count(self):
        return self._count
    def increment(self):
        self._count += 1
    @property
    def root(self):
        return self._item is None and self._count is None
    @property
    def leaf(self):
        return len(self._children) == 0
    @property
    def parent(self):
        return self._parent
    @parent.setter
    def parent(self, value):
        self._parent = value
    @property
    def neighbor(self):
        return self._neighbor
    @neighbor.setter
    def neighbor(self, value):
        self._neighbor = value
    @property
    def children(self):
        return tuple(self._children.itervalues())
    def disp(self, ind=1):
        if self._item != None:
            if self._parent and self._parent._first == True:
                print(" (", end="")
                self._parent._first = False
            print(self._item, end="")
            print(":", end="")
            print(self._count, end="")
            print(":", end="")
            parent = self._parent
            p = ''
            while parent != None and parent._item != None: 
                p = p + parent._item
                parent = parent._parent
            print(p[::-1], end="")
            if self._children == {}:
                print("(", end="")
        for child in self._children.values():
            child.disp(ind+1)
            print("),", end="")


result = []
for itemset, support in myFpgrowth(data, MinsupValue):
    result.append((itemset,support))

print()
print("Result set: [", end="")
count = 0
for itemset, support in result:
    print("(", end="")
    print(itemset, end="")
    print(",", support, end="")
    print(")", end="")
    if count + 1 < len(result):
        print(",", end="")
    count += 1
print("]")
print()
    
print("Result table: ")
count = 0
for itemset, support in result:
    print(count, " ", support/len(data), " ", end= " ")
    count += 1
    print(itemset, '     '*(4-3*len(itemset)), len(itemset))
print()
print("END FPGrowthAlg")

1.3. FPGrowth alg
Data:  [['A', 'F'], ['A', 'D', 'E'], ['E', 'G'], ['A', ' C', 'D'], ['A', 'F', 'G'], ['A', 'B', 'C', 'F', 'G'], ['A', 'B', 'C', 'D', 'G'], ['B', 'D', 'G'], ['A', 'C', 'E', 'F', 'G'], ['C', 'D']]
Minsup ratio:  0.3
Minsup value:  3
Unique el's:  ['A', 'B', 'C', 'D', 'E', 'F', 'G']

Start FPGrowthAlg with I and minsup value:  ['A', 'B', 'C', 'D', 'E', 'F', 'G'] ,  3
Add to F: ('A', 7)
Add to F: ('G', 6)
Add to F: ('D', 5)
Add to F: ('C', 4)
Add to F: ('F', 4)
Add to F: ('B', 3)
Add to F: ('E', 3)
Add to F: (' C', 1)
 
Start R_FPGrowthAlg with Data, Add and gen_prefix:  [['A', 'F'], ['A', 'D', 'E'], ['E', 'G'], ['A', ' C', 'D'], ['A', 'F', 'G'], ['A', 'B', 'C', 'F', 'G'], ['A', 'B', 'C', 'D', 'G'], ['B', 'D', 'G'], ['A', 'C', 'E', 'F', 'G'], ['C', 'D']] , None
Result sorting unique:  {'A': 7, 'G': 6, 'D': 5, 'C': 4, 'F': 4, 'B': 3, 'E': 3, ' C': 1}
FP Tree: root: (A:7: (F:1:A(),D:2:A (E:1:AD(), C:1:AD(),),G:4:A (F:1:AG(),C:2:AG (F:2:AGC (B:1:AGCF(),E:1:AGCF(),),),D:1:AG (